In [20]:
from pathlib import Path
import xarray
import datatree
datadir = Path.home() / "Downloads" # Path.home() makes the code work for different users  

In [21]:
def read_vnp03(filename):    
    # Open the file & make it a dataset
    vnp03 = datatree.open_datatree(datadir / filename) 
    list(vnp03) 

    # Access the relavant variables 
    height_ds = vnp03['geolocation_data']['height']  
    lwm_ds = vnp03['geolocation_data']['land_water_mask']   
    lat_ds = vnp03['geolocation_data']['latitude']   
    lon_ds = vnp03['geolocation_data']['longitude']  
    senazi_ds = vnp03['geolocation_data']['sensor_azimuth']  
    senzen_ds = vnp03['geolocation_data']['sensor_zenith']   
    solazi_ds = vnp03['geolocation_data']['solar_azimuth']   
    solzen_ds = vnp03['geolocation_data']['solar_zenith']   

    # Apply scaling factors/offsets 
    height_scaled = height_ds * 1
    lwm_scaled = lwm_ds
    lat_scaled = lat_ds
    lon_scaled = lon_ds
    senazi_scaled = senazi_ds * 0.01
    senzen_scaled = senzen_ds * 0.01
    solazi_scaled = solazi_ds * 0.01
    solzen_scaled = solzen_ds * 0.01 

    # Extra: Read the long_names
    height_long = height_ds.attrs['long_name']
    lwm_long = lwm_ds.attrs['long_name']
    lat_long = lat_ds.attrs['long_name']
    lon_long = lon_ds.attrs['long_name'] 
    senazi_long = senazi_ds.attrs['long_name']
    senzen_long = senzen_ds.attrs['long_name']
    solazi_long = solazi_ds.attrs['long_name']
    solzen_long = solzen_ds.attrs['long_name'] 


In [29]:
#def map_vnp03(): 


<xarray.Dataset>
Dimensions:          (number_of_lines: 3232, number_of_pixels: 3200)
Dimensions without coordinates: number_of_lines, number_of_pixels
Data variables:
    latitude         (number_of_lines, number_of_pixels) float32 ...
    longitude        (number_of_lines, number_of_pixels) float32 ...
    height           (number_of_lines, number_of_pixels) float32 ...
    range            (number_of_lines, number_of_pixels) float32 ...
    sensor_azimuth   (number_of_lines, number_of_pixels) float32 ...
    sensor_zenith    (number_of_lines, number_of_pixels) float32 ...
    solar_azimuth    (number_of_lines, number_of_pixels) float32 ...
    solar_zenith     (number_of_lines, number_of_pixels) float32 ...
    land_water_mask  (number_of_lines, number_of_pixels) float32 ...
    water_present    (number_of_lines, number_of_pixels) float32 ...
    quality_flag     (number_of_lines, number_of_pixels) uint8 ...

In [22]:
read_vnp03("VNP03MOD.A2020166.1342.002.2021125191832.nc")